In [10]:
import tensorflow as tf
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
AUTOTUNE = tf.data.experimental.AUTOTUNE

batch_size = 3
IMG_HEIGHT = 500
IMG_WIDTH  = 500

Bat_Computer = True
if(Bat_Computer):
    projects_path = '/home/ibrahim/Projects/Self_Maneuvering_UGV/'

else:
    projects_path = '/home/ibrahim/Projects/Self_Maneuvering_UGV/'


images_path = '/home/ibrahim/Projects/Datasets/FYP_Self_Driving_UGV/AirSim/Take 1-20210123T061655Z-001/Take 1/images/'

csv_path    = '/home/ibrahim/Projects/Datasets/FYP_Self_Driving_UGV/AirSim/Take 1-20210123T061655Z-001/Take 1/airsim_rec.txt'

models_path = '/home/ibrahim/Projects/Self_Maneuvering_UGV/TF'

In [11]:
csv_file = pd.read_csv(csv_path, sep="\t")
# csv_file.columns = ["a", "b", "c", "etc."]
print(csv_file.columns)
print(csv_file.head(3)['ImageFile'])

image_names =  images_path +  csv_file['ImageFile'] 
steering_angles = csv_file['Steering']
print(image_names[0])
print(steering_angles[0])


Index(['TimeStamp', 'POS_X', 'POS_Y', 'POS_Z', 'Q_W', 'Q_X', 'Q_Y', 'Q_Z',
       'Throttle', 'Steering', 'Brake', 'Gear', 'Handbrake', 'RPM', 'Speed',
       'ImageFile'],
      dtype='object')
0    img__0_1609577933677259300.png
1    img__0_1609577933755629500.png
2    img__0_1609577933852112100.png
Name: ImageFile, dtype: object
/home/ibrahim/Projects/Datasets/FYP_Self_Driving_UGV/AirSim/Take 1-20210123T061655Z-001/Take 1/images/img__0_1609577933677259300.png
0.0


In [12]:
steering_angles.shape

(1094,)

In [13]:
# Not Using prefetching here && TF sucks !!

dataset = tf.data.Dataset.from_tensor_slices((image_names,steering_angles))

def load_images(filename, target):
    img = tf.io.read_file(filename)
    print(img)
    img = tf.image.decode_jpeg(img, channels=3)
    # img = img[150:][150:]
    # img = img.numpy()

    return img, target

def image_augmentations(image, label):
    # Change to grayscale:
    max_delta = 0.8
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.random_brightness(image, max_delta, seed=None)
    image = tf.image.resize(image, size=(IMG_HEIGHT,IMG_WIDTH))
    return image, label

# Note: DO NOT USE MAPPING FOR ANY OTHER CHILD SETS CREATED

dataset = (
  dataset
  .map(load_images,num_parallel_calls=AUTOTUNE) #can use parrallelism as second output
  .map(image_augmentations,num_parallel_calls=AUTOTUNE) 
  # .shuffle(1000)  #Input must be = the number of total imgs
)
shape = 0
for image, label in dataset.take(5):
  # print("Image shape: ", image.numpy().shape)
  # print("Label: ", label.numpy())
  image = image.numpy()
  cv2.imshow('frame',image)
  shape = image.shape
  cv2.waitKey(0)

print(shape)
cv2.destroyAllWindows()


Tensor("ReadFile:0", shape=(), dtype=string)
(500, 500, 1)


In [14]:
# Improve input pipeline performance by using prefetching etc

train_ds = dataset
 #change according to hardware auqat

# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

def configure_for_performance(ds):
  ds = ds.cache()
#   ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds


train_ds = configure_for_performance(train_ds)



In [15]:
image_batch, label_batch = next(iter(train_ds))

# for i in range(3):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].numpy().astype("uint8"), cmap = "gray")
#   label = label_batch[i]
# #   plt.title(class_names[label])
#   plt.axis("off")



In [16]:

os.chdir(models_path)
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras import Sequential
from tensorflow.keras import datasets, layers, models

custom_model_2 = models.Sequential()
custom_model_2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)))
custom_model_2.add(layers.MaxPooling2D((2, 2)))
custom_model_2.add(layers.Conv2D(64, (3, 3), activation='relu'))
custom_model_2.add(layers.MaxPooling2D((2, 2)))
custom_model_2.add(layers.Conv2D(64, (3, 3), activation='relu'))
custom_model_2.add(layers.Flatten())
custom_model_2.add(layers.Dense(64, activation='relu'))
custom_model_2.add(layers.Dense(1))

custom_model_2.compile(
  optimizer='adam',
  loss='mse',
  )

custom_model_2.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 498, 498, 32)      320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 249, 249, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 247, 247, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 123, 123, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 121, 121, 64)      36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 937024)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [17]:
custom_model_2.fit(
  train_ds,
  epochs=1
)

365/365 [==============================] - 371s 1s/step - loss: 276520.4375
